## Econometric UNFCCC Green Cities Commitment Analysis: USA
### Data Preparation

Data includes the following:
1. UNFCCC data on cities' commitments, actions, etc
2. 2020 Election results by state

In [95]:
import pandas as pd
import numpy as np
import re

#### 1. UNFCCC Data

In [96]:
UNFCCC = pd.read_csv('UNFCCC.csv')

In [97]:
UNFCCC = UNFCCC[(UNFCCC['country'] == 'United States of America') & (UNFCCC['organizationType'] == 'City') & (UNFCCC['actorProperties_population'] >= 1000)]  # Filter for cities in the US with populations over 1000 
UNFCCC['Date'] = pd.to_datetime(UNFCCC['Date'], format = '%Y-%m-%d')  # format date as datetime object
UNFCCC = UNFCCC.sort_values(by = 'Date', ascending = False)
UNFCCC = UNFCCC.drop_duplicates(subset = 'publicId', keep = 'first')  # keep only most recent observation of a city
UNFCCC.reset_index(inplace = True)  # Reset index
UNFCCC.drop(['index','Date','id','accountingYear','organizationType'], axis = 1, inplace = True)  # drop unneeded columns
UNFCCC.rename(columns = {'actorProperties_population':'population'}, inplace = True)

In [98]:
# Generate City and State Columns from OrganisationName
UNFCCC['city'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[0].strip())
UNFCCC['state'] = UNFCCC['organizationName'].apply(lambda x: x.split(',')[1].strip() if len(x.split(',')) == 2 else np.NaN)   

# Clean City names for better joining
UNFCCC['city'] = UNFCCC['city'].apply(lambda x: re.sub(r'City of |City Of |Town Of |Township Of |City and County of |Borough Of |(Town)', '', x))

UNFCCC = UNFCCC[['city','state','population','hasCommitments','hasActionsUndertaken',
                'hasEmissionInventory','hasInitiativeParticipations','hasImpact','hasMitigations',
                'hasAdaptations','hasRiskAssessments','hasClimateActionPlans','hasFinanceActions']]

In [101]:
# Remove the following irregular / nonconforming observations:
    # Metropolitan Council, Twin Cities  --> covers bistate area and Minneapolis is already included
    # Chicago Metropolitan Mayors Caucus  --> Chicago is already included
    # Metropolitan Washington Council of Governments (COG)  --> covers bistate area and District of Columbia is already included
    # Mid-America Regional Council  --> covers bistate area and Kansas City is already included
    # San Francisco/Bay Area Air Quality Management District  --> San Francisco is already included

cities_to_remove = ['Metropolitan Council', 'Chicago Metropolitan Mayors Caucus', 'Metropolitan Washington Council of Governments (COG)', 'Mid-America Regional Council', 'San Francisco/Bay Area Air Quality Management District’S']
UNFCCC = UNFCCC[~UNFCCC['city'].isin(cities_to_remove)]


# Make the following alterations to irregular / nonconforming observations:
    # Metropolitan Government of Nashville and Davidson County, TN  --> rename to Nashville
    # Durham --> add North Carolina as state

UNFCCC.loc[UNFCCC['city'] == 'Metropolitan Government of Nashville and Davidson County', 'city'] = 'Nashville'
UNFCCC.loc[UNFCCC['city'] == 'Durham', 'state'] = 'NC'

In [102]:
UNFCCC

,city,state,population,hasCommitments,hasActionsUndertaken,hasEmissionInventory,hasInitiativeParticipations,hasImpact,hasMitigations,hasAdaptations,hasRiskAssessments,hasClimateActionPlans,hasFinanceActions
0,Village of Park Forest,IL,21261.0,True,True,False,False,False,True,True,True,True,False
1,Emeryville,CA,12905.0,True,True,True,True,False,True,True,True,True,False
2,Grand Rapids,MI,199417.0,True,True,False,True,False,True,True,True,False,False
3,Fremont,CA,230504.0,True,True,True,True,False,True,True,True,True,False
4,Fort Worth,TX,918915.0,False,True,False,False,False,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,Orange County,CA,146027.0,False,False,False,True,False,False,False,False,False,False
230,King County,WA,2117125.0,False,True,False,True,False,False,False,False,False,True
231,Milwaukie,OR,20291.0,False,True,False,False,False,False,True,True,False,False
232,Brisbane,CA,4603.0,False,False,False,True,False,False,False,False,False,False


#### 2. 2020 Election Results by State
Wikipedia Contributors (2019). 2020 United States presidential election. [online] Wikipedia. Available at: https://en.wikipedia.org/wiki/2020_United_States_presidential_election.

In [40]:
ElectionbyState = pd.read_csv('ElectionbyState.csv')

In [41]:
RedBlue = lambda row: 'blue' if row['Biden/Harris_Democratic_EV'] > row['Trump/Pence_Republican_EV'] else 'red'  # Assign blue for a state if it had more democratic electoral college votes
ElectionbyState['redBlueState'] = ElectionbyState.apply(RedBlue, axis = 1)
ElectionbyState = ElectionbyState[['state_abb','redBlueState']]
ElectionbyState.rename(columns = {'state_abb':'state'}, inplace = True)

In [43]:
ElectionbyState.head()

,state,redBlueState
0,AL,red
1,AK,red
2,AZ,blue
3,AR,red
4,CA,blue
